In [9]:
def write_solution(filename ,multiply):
    with open(filename) as f:
        content = f.readlines()

    content = [x.strip() for x in content]

    t_array = []
    sig_array = []

    for index in range(0, len(content)):
        splitted = content[index].rsplit(' ')
        t_array.append(float(splitted[0])/1e9)
        val = float(splitted[1]) * multiply
        if (val > 0):
            sig_array.append(np.log(val))
        else:
            sig_array.append(np.log(np.abs(val)))
        

    def build_a(t_arr):
        if isinstance(t_arr, list):
            if len(t_arr) >= 7:
                a = []
                for index in range(len(t_arr)):
                    t = t_arr[index]
                    a.append([1, t**(-1), t**(-1.0/3), t**(1.0/3), t, t**(5.0/3), np.log(t)])
                return a
            else:
                pass
        else:
            pass

    a = build_a(t_array)
    b = sig_array

    x = []; y = []

    # for index in range(0, 24-7):
    #     x.append(t_array[index])
    #     y.append(np.linalg.solve(a[index:index + 7], b[index:index + 7]))

    # def plot_solution(t_array, solution, block):
    #     if (block != True):
    #         block = False
    #     a1_array = []; b1_array = []
    #     for t_index in range(0, len(t_array)):
    #         current_a1 = 0
    #         for a_index in range(0, len(solution)):
    #             current_a1 += solution[a_index] * a[t_index][a_index]
    #         a1_array.append(current_a1)
    #         b1_array.append(b[t_index])
    #     pylab.figure()
    #     line1, = pylab.plot(t_array, a1_array, label="A", linestyle="--")
    #     line2, = pylab.plot(t_array, b1_array, label="B")    
    #     pylab.legend(handles=[line1, line2], loc=1)
    #     pylab.show(block=block)

    x_lstsq_full, _, _, _ =np.linalg.lstsq(a, b, rcond=1)
    # plot_solution(t_array, x_lstsq, False)
    # -6.461089e+02-2.855431e+02 4.604533e+03-3.632646e+03                      
    #  1.326413e+02-5.591785e+00 2.405325e+03     
    # plot_solution(t_array, [-6.461089e2,-2.855431e2,4.604533e3,-3.632646e3,1.326413e2, -5.591785, 2.405325e3], True)
    # x_lstsq_part, _, _, _ =np.linalg.lstsq(a[2:20], b[2:20], rcond=1)
    # plot_solution(t_array, x_lstsq, True)
    # print(x_lstsq_full, file=f)

    def save_to_file(x_lstsq_full, filename, elem1, elem2):
        save_file = open(filename, 'a')
        print >>save_file,"1"
        print >>save_file, "     {:>5}{:>5}                            cleaw     0.00000e+00          ".format(elem1, elem2)
        prettyfied = []
        for index in range(len(x_lstsq_full)):
            prettyfied.append("{0:1.6e}".format(x_lstsq_full[index]))
        print >>save_file, "{:>13}{:>13}{:>13}{:>13}                      ".format(prettyfied[0], prettyfied[1],prettyfied[2],prettyfied[3])
        print >>save_file, "{:>13}{:>13}{:>13}                                   ".format(prettyfied[4], prettyfied[5],prettyfied[6])
        save_file.close()

    def save_to_file_with_p(x_lstsq_full, filename, elem1, elem2):
        save_file = open(filename, 'a')
        print("5", file=save_file)
        print("         p{:>5}{:>5}    p                  cleaw     0.00000e+00          ".format(elem1, elem2), file=save_file)
        prettyfied = []
        for index in range(len(x_lstsq_full)):
            prettyfied.append("{0:1.6e}".format(x_lstsq_full[index]))
        print("{:>13}{:>13}{:>13}{:>13}                      ".format(prettyfied[0], prettyfied[1],prettyfied[2],prettyfied[3]), file=save_file)
        print("{:>13}{:>13}{:>13}                                   ".format(prettyfied[4], prettyfied[5],prettyfied[6]), file=save_file)
        save_file.close()

    names = os.path.splitext(os.path.basename(filename))[0].rsplit('-')
    
    #print(x_lstsq_full)
    save_to_file_with_p(x_lstsq_full, "сollisional-beta-decay/lib", names[0], names[1])

In [10]:
import os
import numpy as np
import sys
sys.path.append('/home/noir/skynet-lib/lib')
from SkyNet import *

def calculate_rprocess(with_my, fileName):
    do_heat = False; do_inv = False; do_screen = False
    with open("r-process/sunet") as f:
        nuclides = [l.strip() for l in f.readlines()]

    # make trajectory
    t0 = 1.0e-3
    tfin = 1.0e9

    nuclib = NuclideLibrary.CreateFromWebnucleoXML(SkyNetRoot
        + "/data/webnucleo_nuc_v2.0.xml")
    print(nuclib.NumNuclides())

    opts = NetworkOptions()
    opts.ConvergenceCriterion = NetworkConvergenceCriterion.Mass
    opts.MassDeviationThreshold = 1.0E-10
    opts.IsSelfHeating = do_heat
    opts.EnableScreening = do_screen
    opts.DisableStdoutOutput = True
    
    helm = HelmholtzEOS(SkyNetRoot + "/data/helm_table.dat")
    if (with_my):
        strongReactionLibrary = REACLIBReactionLibrary("reaclib-wo-reactions",
          ReactionType.Strong, do_inv, LeptonMode.TreatAllAsDecayExceptLabelEC,
          "Strong reactions", nuclib, opts, True, True)
        weakReactionLibrary = REACLIBReactionLibrary("reaclib-wo-reactions",
          ReactionType.Weak, False, LeptonMode.TreatAllAsDecayExceptLabelEC,
          "Weak reactions", nuclib, opts, True, True)
    else:
        strongReactionLibrary = REACLIBReactionLibrary(SkyNetRoot + "/data/reaclib",
            ReactionType.Strong, do_inv, LeptonMode.TreatAllAsDecayExceptLabelEC,
            "Strong reactions", nuclib, opts, True, True)
        weakReactionLibrary = REACLIBReactionLibrary(SkyNetRoot + "/data/reaclib",
            ReactionType.Weak, False, LeptonMode.TreatAllAsDecayExceptLabelEC,
            "Weak reactions", nuclib, opts, True, True)
        
    symmetricFission = REACLIBReactionLibrary("r-process/nfis",
            ReactionType.Strong, False, LeptonMode.TreatAllAsDecayExceptLabelEC,
            "Symmetric neutron induced fission with 0 neutrons emitted",
            nuclib, opts, True, True)
    spontaneousFission = REACLIBReactionLibrary("r-process/sfis",
            ReactionType.Strong, False, LeptonMode.TreatAllAsDecayExceptLabelEC,
            "Spontaneous fission", nuclib, opts, True, True)  
    myLib = REACLIBReactionLibrary("сollisional-beta-decay/lib",
            ReactionType.Weak, False, LeptonMode.TreatAllAsDecayExceptLabelEC,
            "CAP", nuclib, opts, True, True)

    if (with_my):
        reactionLibraries = [strongReactionLibrary, weakReactionLibrary,
                             symmetricFission, spontaneousFission, myLib]
    else:
        reactionLibraries = [strongReactionLibrary, weakReactionLibrary,
                             symmetricFission, spontaneousFission]  

    screen = SkyNetScreening(nuclib)
    net = ReactionNetwork(nuclib, reactionLibraries, helm, screen, opts)

  #lib = net.GetNuclideLibrary()
  #fout = open("sunet", "w")
  #for n in lib.Names():
    #fout.write("%5s\n" % n)
  #fout.close()

    dat = np.loadtxt("r-process/traj_skynet")
    density_vs_time = PiecewiseLinearFunction(dat[:,0], dat[:,2], True)
    temperature_vs_time = PiecewiseLinearFunction(dat[:,0], dat[:,1], True)

    Ye0 = 0.07
    Temp0 = dat[0,1]
    tfinal = 5.0e8

    # initial abundance
    nse = NSE(net.GetNuclideLibrary(), helm, screen)
    nseResult = nse.CalcFromTemperatureAndDensity(Temp0, density_vs_time(t0), Ye0)
    initY = nseResult.Y()
    
    print(len(initY))

    if (do_heat):
        output = net.EvolveSelfHeatingWithInitialTemperature(initY, t0, tfinal,
            Temp0, density_vs_time, fileName)
    else:
        output = net.Evolve(initY, t0, tfinal, temperature_vs_time, density_vs_time, fileName)

index = 1
multiply = 1

onlyfiles = [f for f in os.listdir('сollisional-beta-decay/sigmas') if os.path.isfile(os.path.join('сollisional-beta-decay/sigmas', f))]

if (os.path.exists("сollisional-beta-decay/lib")):
    os.remove("сollisional-beta-decay/lib")
    
for file_index in range(0, len(onlyfiles)):
    write_solution('сollisional-beta-decay/sigmas/' + onlyfiles[file_index], multiply)

calculate_rprocess(True, "r-process/result/WithMy-" + str(index))
calculate_rprocess(False, "r-process/result/WithoutMy-" + str(index))

8230
7836
8230
7836


In [3]:
import pandas as pd

array_of_interesting_nucldies_mother = [
    "cd106",
    "cd108",
    " kr78",
    " kr80",
    "ba130",
    "ba132",
    "er164",
    "xe124",
    "xe126",
    "sn112",
    "sn114",
    "ce136",
    " sr84",
    "te120",
    " se74",
    "hf136"
]

dictionary = {
    "cd": 48,
    "kr": 36,
    "ba": 56,
    "er": 68,
    "xe": 54,
    "sn": 50,
    "ce": 58,
    "sr": 38,
    "te": 43,
    "se": 34,
    "hf": 72
}

experimental_data = {
    "74|34": 0.580,
    "78|36": 0.146,
    "80|36": 0.940,
    "84|38": 0.128,
    "92|42": 0.634,
    "94|42": 0.361,
    "96|44": 0.105,
    "98|44": 0.036,
    "102|46": 0.013,
    "106|48": 0.019,
    "108|48": 0.014,
    "110|48": 0.020,
    "112|50": 0.036,
    "114|50": 0.024,
    "120|52": 5.8e-3,
    "124|54": 7.4e-3,
    "126|54": 6.7e-3,
    "130|56": 4.8e-3,
    "132|56": 4.7e-3,
    "136|58": 2.3e-3,
    "138|58": 3e-3,
    "144|62": 7.4e-3,
    "152|64": 8.4e-4,
    "156|66": 1.9e-4,
    "158|66": 3.3e-4,
    "162|68": 3.1e-4,
    "164|68": 3.6e-3,
    "168|70": 2.4e-4,
    "174|72": 3.1e-4,
    "180|74": 4e-4,
    "184|76": 1.2e-4,
    "190|78": 1.8e-4,
    "196|80": 3.1e-4
}

def compare_h5(fn1, fn2):
    def get_index(store, a, z):
        if (isinstance(store, pd.HDFStore)):
            sa = pd.Series(store.root.A)
            sz = pd.Series(store.root.Z)
            a_arr = sa[sa == a].keys()
            z_arr = sz[sz == z].keys()
            index = a_arr.intersection(z_arr)
            if (len(index) == 1):
                return index[0]
            else:
                print("Element not found, A = {0}, Z={1}".
                               format(a, z))
                return -1
        else:
            raise "Error"

    store1 = pd.io.pytables.HDFStore(fn1)
    store2 = pd.io.pytables.HDFStore(fn2)
    save_file = open("outcome_new", "a")
    
    for index in range(0, len(array_of_interesting_nucldies_mother)):
        # item_for_compare = 'kr78'
        elem_name = array_of_interesting_nucldies_mother[index]
        z = dictionary[''.join(x for x in elem_name if x.isalpha())]        
        # z_compare = dictionary[''.join(x for x in item_for_compare if x.isalpha())]
        a = int(''.join(x for x in elem_name if x.isdigit()))
        # a_compare = int(''.join(x for x in item_for_compare if x.isdigit()))
        elem_index = get_index(store1, a, z)
        if (elem_index == -1):
            continue
        size_t1, _ = store1.root.Y.shape
        y1 = store1.root.Y[size_t1 - 1][elem_index]
        size_t2, _ = store2.root.Y.shape
        y2 = store2.root.Y[size_t2 - 1][elem_index]
        print("{0}, WithMy: {1}, WithoutMy: {2}".format(elem_name, y1, y2))
        print("{0}, {1}, {2}".format(elem_name, y1, y2), file=save_file)
        
    store1.close()
    store2.close()
    save_file.close()

folder_name = 'r-process/result'
for index in range(1, 2):
    compare_h5(folder_name + '/WithMy-' + str(index) + '.h5', 
        folder_name + '/WithoutMy-' + str(index) + '.h5')

cd106, WithMy: 4.353844161456553e-113, WithoutMy: 4.3538289993529215e-113
cd108, WithMy: 1.335488699232695e-22, WithoutMy: 1.3355024383723382e-22
 kr78, WithMy: 4.302574436158129e-99, WithoutMy: 4.302580721551153e-99
 kr80, WithMy: 1.3705629752500308e-22, WithoutMy: 1.3706741878682557e-22
ba130, WithMy: 2.0559363840835507e-136, WithoutMy: 2.056185808464833e-136
ba132, WithMy: 6.930894713582514e-141, WithoutMy: 6.945219630959364e-141
er164, WithMy: 0.0, WithoutMy: 0.0
xe124, WithMy: 1.2702804892636204e-130, WithoutMy: 1.2704106687511601e-130
xe126, WithMy: 6.567456902363498e-123, WithoutMy: 6.567661308877963e-123
sn112, WithMy: 1.3078299301511546e-114, WithoutMy: 1.3078419977345317e-114
sn114, WithMy: 2.608454846976493e-32, WithoutMy: 2.608506273987205e-32
ce136, WithMy: 3.706069264347549e-145, WithoutMy: 3.706220185837257e-145
 sr84, WithMy: 2.1887739357587232e-95, WithoutMy: 2.18880485288546e-95
te120, WithMy: 3.202993942564247e-234, WithoutMy: 3.203517648447467e-234
 se74, WithMy: 3.